In [1]:
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession \
        .builder \
        .appName("Python Spark stream tutorials") \
        .config("spark.some.config.option", "some-value") \
        .getOrCreate()

In [2]:
spark

## 고객별 가장 많이 소비한 날 확인

In [3]:
# 시계열 데이터
staticDataFrame = spark.read.format("csv")\
    .option("header","true")\
    .option("inferSchema", "true")\
    .load("../data/retail-data/by-day/*.csv")

In [4]:
# DB 테이블 생성
staticDataFrame.createOrReplaceTempView("retail_data")
staticSchema = staticDataFrame.schema

In [8]:
staticDataFrame

DataFrame[InvoiceNo: string, StockCode: string, Description: string, Quantity: int, InvoiceDate: string, UnitPrice: double, CustomerID: double, Country: string]

In [9]:
from pyspark.sql.functions import window, col
# 윈도우 함수 적용 일별
staticDataFrame\
    .selectExpr(
    "CustomerId",
    "(UnitPrice * Quantity) as total_cost",
    "InvoiceDate")\
    .groupBy(
    col("CustomerId"), window(col("InvoiceDate"), "1 day"))\
    .sum("total_cost")\
    .show(5)

+----------+--------------------+-----------------+
|CustomerId|              window|  sum(total_cost)|
+----------+--------------------+-----------------+
|   16057.0|{2011-12-05 09:00...|            -37.6|
|   14126.0|{2011-11-29 09:00...|643.6300000000001|
|   13500.0|{2011-11-16 09:00...|497.9700000000001|
|   17160.0|{2011-11-08 09:00...|516.8499999999999|
|   15608.0|{2011-11-11 09:00...|            122.4|
+----------+--------------------+-----------------+
only showing top 5 rows



In [10]:
# 셔플 파티션 수는 셔플 이후에 생성될 파티션 수
# 로컬 모드에서는 많은 익스큐터가 필요하지 않기 때문에 이 값을 5로 설정
spark.conf.set("spark.sql.shuffle.partitions", "5")

In [12]:
# maxFilesPerTrigger 한번에 읽을 파일 수 지정 -> 실제 운영 환경에선 X
streamingDataFrame = spark.readStream\
.schema(staticSchema)\
.option("maxFilesPerTrigger",1)\
.format("csv")\
.option("header", "true")\
.load("../data/retail-data/by-day/*.csv")

In [16]:
# 스트리밍 유형인지 확인 (데이터를 연속적으로 전달하는 데이터 소스라면 true)
streamingDataFrame.isStreaming

True

In [22]:
# 총 판매 금액 계산 코드
# 아래 코드는 지연 연산으로 데이터 플로를 실행하기 위해 스트리밍 액션을 호출해야 함
purchaseByCustomerPerHour = streamingDataFrame\
    .selectExpr(
    "CustomerId",
    "(UnitPrice * Quantity) as total_cost",
    "InvoiceDate")\
    .groupBy(
    col("CustomerId"), window(col("InvoiceDate"), "1 day"))\
    .sum("total_cost")

In [23]:
purchaseByCustomerPerHour.writeStream\
    .format("memory")\
    .queryName("customer_purchases")\
    .outputMode("complete")\
    .start()

In [24]:
spark.sql("""
SELECT *
FROM customer_purchases
ORDER BY 'sum(total_cost)' DESC
""")\
.show(5)

+----------+--------------------+------------------+
|CustomerId|              window|   sum(total_cost)|
+----------+--------------------+------------------+
|   17576.0|{2010-12-13 09:00...|177.35000000000002|
|   15039.0|{2010-12-14 09:00...| 706.2500000000002|
|   16250.0|{2010-12-01 09:00...|            226.14|
|   14594.0|{2010-12-01 09:00...|254.99999999999997|
|   15899.0|{2010-12-06 09:00...|             56.25|
+----------+--------------------+------------------+
only showing top 5 rows



In [25]:
purchaseByCustomerPerHour.writeStream\
    .format("console")\
    .queryName("customer_purchases_2")\
    .outputMode("complete")\
    .start()